In [38]:
import pandas as pd
import openai
import dotenv
from dotenv import load_dotenv
load_dotenv()


dataset = pd.read_csv('supermarket_sales.csv')
dataset.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [39]:
dataset['Rating'].mean()

6.9727

## Criand Assistant ## 

In [56]:
file = openai.files.create(
    file=open('supermarket_sales.csv','rb'),
    purpose='assistants'
)

In [75]:
assistant = openai.beta.assistants.create(
    name='Analista de dados , Machine Learning e LLM ',
    instructions='Fornecer e analiser dados e gerar codigos em python e analisar estatisticas voce vai usar os arquivos .csv \
    informados para fazer as devidas analises extraia o maximo de inforamções possiveis em formato e crie graficos mais apropriados para os dados achados',
    tools=[{'type':'code_interpreter'}],
    tool_resources={'code_interpreter' : {'file_ids': [file.id]}},
    model='gpt-4o'
)

In [76]:
thread = openai.beta.threads.create()

In [77]:
texto_mensagem = 'Faça analise de dados da planilha e traga insights e dados relevantes para tomada de descisão'
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role = 'user',
    content = texto_mensagem
)

In [78]:
## Rodando a mensagem gerada com thread ##

run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome e felippe ele e um usuario premium'
)

In [79]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = openai.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [80]:
run.status

'completed'

In [81]:
if run.status == 'completed':
    messagens = openai.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messagens)

else:
    print('errro',run.status)

SyncCursorPage[Message](data=[Message(id='msg_h0Q4SiskmFjxPnYyG82FYZgF', assistant_id='asst_M2WLhoF0hvLnS1sV5EVxcxBw', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Aqui estão alguns insights iniciais das estatísticas descritivas e da verificação de dados ausentes:\n\n### Estatísticas Descritivas:\n- **Unit Price**: O preço unitário dos produtos varia de 10,08 a 99,96, com uma média de aproximadamente 55,67.\n- **Quantity**: As quantidades vendidas variam de 1 a 10 unidades, com uma média de 5,51 unidades por transação.\n- **Total**: O valor total das vendas oscila de 10,68 a 1042,65, com uma média de aproximadamente 322,97.\n- **Gross Income**: A renda bruta varia de 0,51 a 49,65, com uma média de 15,38.\n\n### Dados Ausentes:\n- Não há valores ausentes em nenhuma das colunas.\n\n### Próximas Etapas:\nVamos explorar algumas questões mais específicas, como a distribuição de vendas por cidade, tipos de clientes, linhas de produtos, e métod

In [82]:
print(messagens.data[0].content[0].text.value)


Aqui estão alguns insights iniciais das estatísticas descritivas e da verificação de dados ausentes:

### Estatísticas Descritivas:
- **Unit Price**: O preço unitário dos produtos varia de 10,08 a 99,96, com uma média de aproximadamente 55,67.
- **Quantity**: As quantidades vendidas variam de 1 a 10 unidades, com uma média de 5,51 unidades por transação.
- **Total**: O valor total das vendas oscila de 10,68 a 1042,65, com uma média de aproximadamente 322,97.
- **Gross Income**: A renda bruta varia de 0,51 a 49,65, com uma média de 15,38.

### Dados Ausentes:
- Não há valores ausentes em nenhuma das colunas.

### Próximas Etapas:
Vamos explorar algumas questões mais específicas, como a distribuição de vendas por cidade, tipos de clientes, linhas de produtos, e métodos de pagamento. Essas análises podem fornecer insights sobre padrões de consumo, desempenho regional e preferências de pagamento, entre outros aspectos. 

Começaremos analisando a distribuição das vendas por cidade e tipo de

## Analisando Passo a Passo ##

In [83]:
run_steps = openai.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [84]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')


=== Step: message_creation

=== Step: tool_calls
-----
import pandas as pd

# Load the spreadsheet to inspect its content
file_path = '/mnt/data/file-BCHdL3td8vyWfzaUgzaSZ6'
data = pd.ExcelFile(file_path)

# Display the sheet names to get an overview of the content
sheet_names = data.sheet_names
sheet_names
-----
Result

=== Step: message_creation

=== Step: tool_calls
-----
import mimetypes

# Determine the file type
file_type, _ = mimetypes.guess_type(file_path)
file_type
-----
Result

=== Step: message_creation

=== Step: tool_calls
-----
# Try loading the file as a CSV
try:
    data = pd.read_csv(file_path)
    data.head()
except Exception as e:
    str(e)
-----
Result

=== Step: message_creation

=== Step: tool_calls
-----
# Read the beginning of the file to help identify its format
with open(file_path, 'rb') as file:
    file_start = file.read(256)

file_start[:100]  # Show the first 100 bytes for inspection
-----
Result

=== Step: message_creation

=== Step: tool_calls
-----
# 

## Potando Grafico ##


In [85]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
    
    if step.step_details.type == 'message_creation':
        message = openai.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        
        if message.content[0].type == 'text':
            print(message.content[0].text.value)
        
        if message.content[0].type == 'image_file':
            file_id = message.content[0].image_file.file_id
            image_data = openai.files.content(file_id)
            
            # Save the image to a file
            with open(f'{file_id}.png', 'wb') as f:
                f.write(image_data.read())
                print(f'Imagem {file_id} salva')
            
            # Use matplotlib to display the image
            import matplotlib.pyplot as plt
            import matplotlib.image as mpimg
            
            # Read the image from the saved file
            img = mpimg.imread(f'{file_id}.png')
            
            # Create a figure to display the image
            fig, ax = plt.subplots()
            ax.set_axis_off()  # Hide axis
            ax.imshow(img)  # Show image
            plt.show()



=== Step: message_creation
Para começar, vou abrir e examinar o conteúdo do arquivo que você enviou. Isso me ajudará a compreender a estrutura dos dados e identificar quais análises podem ser relevantes. Vamos começar carregando o arquivo e inspecionando suas primeiras linhas.

=== Step: tool_calls
-----
import pandas as pd

# Load the spreadsheet to inspect its content
file_path = '/mnt/data/file-BCHdL3td8vyWfzaUgzaSZ6'
data = pd.ExcelFile(file_path)

# Display the sheet names to get an overview of the content
sheet_names = data.sheet_names
sheet_names
-----
Result

=== Step: message_creation
Parece que houve um problema ao tentar carregar o arquivo. Isso pode ocorrer quando o arquivo não é um formato Excel suportado nativamente. Vamos tentar outro método para abrir o arquivo, possivelmente como um arquivo CSV, se for o caso. Vou verificar o tipo de arquivo para entender seu formato.

=== Step: tool_calls
-----
import mimetypes

# Determine the file type
file_type, _ = mimetypes.gues